<a href="https://colab.research.google.com/github/Trading-com-Dados/tutoriais_publicos/blob/main/20220915_Live_Tipos_de_Join_Trading_com_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

---
# **Integrando diferentes bases de dados com Pandas / Python**
## *Tipos de join na função merge*
---

**Função <u>Join</u> muito utilizada em SQL**

**Função <u>PROCV / VLOOKUP</u> no Excel**

<img src="https://i2.wp.com/www.datascienceexamples.com/wp-content/uploads/2019/12/join_types.png?resize=1024%2C960&ssl=1" width="600" align="left"/>


<img src="https://i0.wp.com/www.datascienceexamples.com/wp-content/uploads/2019/12/joins_with_pandas.png?resize=1024%2C554&ssl=1" width="600" align="left"/>

# **1. Estudo de caso #1: Juntar base de dados de dividendos com preço de fechamento**

In [24]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import yfinance as yf

import pandas as pd

In [26]:
dividendos_PETR4 = yf.Ticker('PETR4.SA').dividends
dividendos_PETR4.head()

Date
2005-04-01    0.133288
2005-07-01    0.250663
2006-01-02    0.259000
2006-04-04    0.132700
2006-11-01    0.500750
Name: Dividends, dtype: float64

In [27]:
fechamento_PETR4 = yf.download('PETR4.SA', period='max')['Close']
fechamento_PETR4.head()

[*********************100%***********************]  1 of 1 completed


Date
2000-01-03    5.875
2000-01-04    5.550
2000-01-05    5.494
2000-01-06    5.475
2000-01-07    5.500
Name: Close, dtype: float64

In [28]:
df_join_dividend_close = pd.merge(dividendos_PETR4, fechamento_PETR4, how='left', left_on=dividendos_PETR4.index, right_on=fechamento_PETR4.index)
df_join_dividend_close.head()

,key_0,Dividends,Close
0,2005-04-01,0.133288,12.98750
1,2005-07-01,0.250663,13.52375
2,2006-01-02,0.259000,19.04500
3,2006-04-04,0.132700,21.64500
4,2006-11-01,0.500750,21.22500


# **2. Estudo de caso #2: Comparar as cotações entre yfinance e trading view**

In [29]:
!pip install tvdatafeed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
from tvDatafeed import TvDatafeed, Interval

import pandas as pd
from datetime import date

Baixar dados do Yahoo Finance

In [31]:
itub4_yf = yf.download('ITUB4.SA', period='max')
itub4_yf.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-12-21,2.815505,2.988535,2.815174,2.960552,1.807515,74224
2000-12-22,2.930748,2.930748,2.814843,2.832229,1.729169,23312
2000-12-25,2.832229,2.832229,2.832229,2.832229,1.729169,0
2000-12-26,2.816664,2.834712,2.790006,2.831401,1.728664,7851
2000-12-27,2.881074,2.927437,2.831401,2.927437,1.787296,26996


Estrutura de código para baixar dados Trading View. É possível inserir usuário e senha para maior lista de ativos disponíveis

Limite de dados costuma ser aproximadamente 5000 "barras" (varia com o timeframe)

In [32]:
tv = TvDatafeed()

itub4_tv = tv.get_hist(symbol='ITUB4',exchange='BMFBOVESPA', interval=Interval.in_daily, n_bars=10000)

In [33]:
itub4_tv.head()

,symbol,open,high,low,close,volume
datetime,,,,,,
2000-01-03 12:00:00,BMFBOVESPA:ITUB4,0.373557,0.386764,0.372614,0.372614,9252311.0
2000-01-04 12:00:00,BMFBOVESPA:ITUB4,0.372614,0.372614,0.360351,0.360351,9050652.0
2000-01-05 12:00:00,BMFBOVESPA:ITUB4,0.350917,0.358464,0.343371,0.350917,38475633.0
2000-01-06 12:00:00,BMFBOVESPA:ITUB4,0.350917,0.355634,0.348087,0.355634,49248315.0
2000-01-07 12:00:00,BMFBOVESPA:ITUB4,0.362237,0.391480,0.360351,0.377330,20632104.0


Faremos o join baseando-se nas datas dos preços OHLC. Entretanto vemos que elas não estão no mesmo formato

Podemos converter os índices para o formato ANO/MÊS/DIA sem incluir as horas

In [34]:
itub4_yf.index = itub4_yf.index.date
itub4_yf.head()

,Open,High,Low,Close,Adj Close,Volume
2000-12-21,2.815505,2.988535,2.815174,2.960552,1.807515,74224
2000-12-22,2.930748,2.930748,2.814843,2.832229,1.729169,23312
2000-12-25,2.832229,2.832229,2.832229,2.832229,1.729169,0
2000-12-26,2.816664,2.834712,2.790006,2.831401,1.728664,7851
2000-12-27,2.881074,2.927437,2.831401,2.927437,1.787296,26996


In [35]:
itub4_tv.index = itub4_tv.index.date
itub4_tv.head()

,symbol,open,high,low,close,volume
2000-01-03,BMFBOVESPA:ITUB4,0.373557,0.386764,0.372614,0.372614,9252311.0
2000-01-04,BMFBOVESPA:ITUB4,0.372614,0.372614,0.360351,0.360351,9050652.0
2000-01-05,BMFBOVESPA:ITUB4,0.350917,0.358464,0.343371,0.350917,38475633.0
2000-01-06,BMFBOVESPA:ITUB4,0.350917,0.355634,0.348087,0.355634,49248315.0
2000-01-07,BMFBOVESPA:ITUB4,0.362237,0.391480,0.360351,0.377330,20632104.0


Agora que os índices estão num mesmo formato de data, podemos fazer o join baseado nos índices

Usaremos o **"INNER"** como tipo de join, já que nos interessa apenas os dados que representam a interseção entre os conjuntos de dados

É importante ressaltar que nesse caso a coluna key_0 (que serviu para unir os dois dataframes) não está no formato de data

In [36]:
df_join_itub4_inner = pd.merge(itub4_yf, itub4_tv, how='inner', left_on=itub4_yf.index, right_on=itub4_tv.index)
df_join_itub4_inner

,key_0,Open,High,Low,Close,Adj Close,Volume,symbol,open,high,low,close,volume
0,2000-12-21,2.815505,2.988535,2.815174,2.960552,1.807515,74224,BMFBOVESPA:ITUB4,0.413176,0.438647,0.413176,0.432043,13024374.0
1,2000-12-22,2.930748,2.930748,2.814843,2.832229,1.729169,23312,BMFBOVESPA:ITUB4,0.430156,0.430156,0.413176,0.416007,4100486.0
2,2000-12-26,2.816664,2.834712,2.790006,2.831401,1.728664,7851,BMFBOVESPA:ITUB4,0.420723,0.423553,0.416950,0.422610,1377071.0
3,2000-12-27,2.881074,2.927437,2.831401,2.927437,1.787296,26996,BMFBOVESPA:ITUB4,0.430156,0.436760,0.422610,0.434873,4741558.0
4,2000-12-28,2.910879,3.129443,2.864351,3.063211,1.870191,77425,BMFBOVESPA:ITUB4,0.434873,0.467890,0.428270,0.457513,13610196.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5378,2022-09-16,26.600000,26.870001,26.459999,26.680000,26.680000,43782000,BMFBOVESPA:ITUB4,26.600000,26.870001,26.459999,26.680000,43782000.0
5379,2022-09-19,26.610001,27.799999,26.580000,27.680000,27.680000,41063200,BMFBOVESPA:ITUB4,26.610001,27.799999,26.580000,27.680000,41063200.0
5380,2022-09-20,27.820000,28.660000,27.770000,28.600000,28.600000,62939000,BMFBOVESPA:ITUB4,27.820000,28.660000,27.770000,28.600000,62939000.0
5381,2022-09-21,28.600000,28.680000,28.180000,28.299999,28.299999,48098500,BMFBOVESPA:ITUB4,28.600000,28.680000,28.180000,28.299999,48098500.0


E se eu quisesse, usando o join, juntar ambos os dados, mesmo que a data esteja disponível em apenas um dos dataframes: **"OUTER"**

Veja nesse exemplo que em 2000-12-25 (Natal), a yf trouxe valores das cotações (mesmo que OHLC sendo igual), e a tv possui uma cobertura temporal maior (final do dataframe)

In [37]:
df_join_itub4_outer = pd.merge(itub4_yf, itub4_tv, how='outer', left_on=itub4_yf.index, right_on=itub4_tv.index)
df_join_itub4_outer

,key_0,Open,High,Low,Close,Adj Close,Volume,symbol,open,high,low,close,volume
0,2000-12-21,2.815505,2.988535,2.815174,2.960552,1.807515,74224.0,BMFBOVESPA:ITUB4,0.413176,0.438647,0.413176,0.432043,13024374.0
1,2000-12-22,2.930748,2.930748,2.814843,2.832229,1.729169,23312.0,BMFBOVESPA:ITUB4,0.430156,0.430156,0.413176,0.416007,4100486.0
2,2000-12-25,2.832229,2.832229,2.832229,2.832229,1.729169,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-12-26,2.816664,2.834712,2.790006,2.831401,1.728664,7851.0,BMFBOVESPA:ITUB4,0.420723,0.423553,0.416950,0.422610,1377071.0
4,2000-12-27,2.881074,2.927437,2.831401,2.927437,1.787296,26996.0,BMFBOVESPA:ITUB4,0.430156,0.436760,0.422610,0.434873,4741558.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5698,2000-12-18,NaN,NaN,NaN,NaN,NaN,NaN,BMFBOVESPA:ITUB4,0.422610,0.437703,0.422610,0.428270,15752054.0
5699,2000-12-19,NaN,NaN,NaN,NaN,NaN,NaN,BMFBOVESPA:ITUB4,0.428270,0.431100,0.422610,0.427326,10920760.0
5700,2000-12-20,NaN,NaN,NaN,NaN,NaN,NaN,BMFBOVESPA:ITUB4,0.420723,0.422610,0.408460,0.408460,6503412.0
5701,2012-02-22,NaN,NaN,NaN,NaN,NaN,NaN,BMFBOVESPA:ITUB4,8.645534,8.678487,8.509013,8.551382,20490442.0


# **3. Estudo de caso #3: Explorando outros parâmetros do merge**

In [38]:
vale3 = yf.download('VALE3.SA', period='max')
azul4 = yf.download('AZUL4.SA', period='max')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [39]:
vale3.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,3.500000,3.542500,3.500000,3.500000,1.707494,585600
2000-01-04,3.466666,3.474166,3.416666,3.416666,1.666839,782400
2000-01-05,3.375000,3.416666,3.375000,3.416666,1.666839,1876800
2000-01-06,3.416666,3.500000,3.416666,3.416666,1.666839,792000
2000-01-07,3.458333,3.559166,3.458333,3.541666,1.727822,5347200


In [40]:
azul4.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-04-11,21.809999,22.980000,21.770000,22.400000,22.400000,13060500
2017-04-12,22.450001,22.990000,22.299999,22.959999,22.959999,1745900
2017-04-13,22.969999,23.010000,22.549999,22.990000,22.990000,549600
2017-04-17,23.000000,23.000000,22.620001,22.850000,22.850000,249300
2017-04-18,22.820000,24.790001,22.600000,23.910000,23.910000,1125100


Transformar o índice em uma nova coluna chamada "Data"

In [41]:
vale3['Data'] = vale3.index
azul4['Data'] = azul4.index
azul4.head()

,Open,High,Low,Close,Adj Close,Volume,Data
Date,,,,,,,
2017-04-11,21.809999,22.980000,21.770000,22.400000,22.400000,13060500,2017-04-11
2017-04-12,22.450001,22.990000,22.299999,22.959999,22.959999,1745900,2017-04-12
2017-04-13,22.969999,23.010000,22.549999,22.990000,22.990000,549600,2017-04-13
2017-04-17,23.000000,23.000000,22.620001,22.850000,22.850000,249300,2017-04-17
2017-04-18,22.820000,24.790001,22.600000,23.910000,23.910000,1125100,2017-04-18


Neste exemplo as datas não se sobrepõe, mas o nome das colunas é o mesmo. Vamos juntar apenas a coluna 'Adj Close' de cada um, baseado na coluna 'Data' e definir um sufixo para sabermos diferenciá-las

In [43]:
df_vale3_azul4 = pd.merge(vale3[['Adj Close', 'Data']], azul4[['Adj Close', 'Data']], how='inner', on='Data', suffixes=['_vale3', '_azul3'])
df_vale3_azul4

,Adj Close_vale3,Data,Adj Close_azul3
0,19.192825,2017-04-11,22.400000
1,18.362431,2017-04-12,22.959999
2,18.362431,2017-04-13,22.990000
3,18.369127,2017-04-17,22.850000
4,17.819994,2017-04-18,23.910000
...,...,...,...
1353,68.250000,2022-09-16,16.799999
1354,70.459999,2022-09-19,17.750000
1355,69.449997,2022-09-20,17.870001
1356,68.449997,2022-09-21,17.490000
